In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.spatial import Voronoi


import tyssue

from tyssue.core.objects import Epithelium
from tyssue.geometry import bulk_geometry as bgeom
from tyssue.core.generation import (data_dicts, make_df,
                                    hexa_grid3d, from_3d_voronoi)
import vispy as vp
from vispy import app, gloo, visuals, scene
from vispy.geometry import MeshData


### Scatter plots of the cell centers

```python
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*grid.T, c='k', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()
```

In [2]:
grid = hexa_grid3d(16, 14, 3)

datasets = from_3d_voronoi(Voronoi(grid))
eptm = Epithelium('start3D', datasets)

gspecs = eptm.set_geom(bgeom)
bgeom.update_all(eptm)
eptm.cell_df.head()

,z,num_faces,vol,x,is_alive,y,area
cell,,,,,,,
0,0.6875,4,0.000000,0.75,1,-0.625,0
1,0.6875,8,0.052083,1.50,1,-0.625,0
2,0.6875,8,0.052083,2.50,1,-0.625,0
3,0.6875,8,0.052083,3.50,1,-0.625,0
4,0.6875,8,0.052083,4.50,1,-0.625,0


In [3]:
eptm.cell_df.head()

,z,num_faces,vol,x,is_alive,y,area
cell,,,,,,,
0,0.6875,4,0.000000,0.75,1,-0.625,0
1,0.6875,8,0.052083,1.50,1,-0.625,0
2,0.6875,8,0.052083,2.50,1,-0.625,0
3,0.6875,8,0.052083,3.50,1,-0.625,0
4,0.6875,8,0.052083,4.50,1,-0.625,0


In [4]:
bounds = [[-0.1, 2], [0, 2], [0, 10]]

je_out = eptm.cut_out(bounds)

eptm.remove(je_out)

In [7]:
vertices, faces, face_mask = eptm.triangular_mesh(eptm.coords)

canvas = scene.SceneCanvas(keys='interactive', show=True)

grid = canvas.central_widget.add_grid()
view = grid.add_view(0, 1)
#view = canvas.central_widget.add_view()
view.camera =  'turntable'
view.camera.aspect = 1


view.bgcolor = vp.color.Color('#aaaaaa')



mesh = vp.scene.visuals.Mesh(vertices=vertices,
                             faces=faces, color=[0.2, 0.5, 0.1, 0.8])

wire_pos = vertices[eptm.Nc:].copy()


wire = vp.scene.visuals.Line(pos=wire_pos, 
                             connect=faces[:, :2] - eptm.Nc,
                             color=[0.1, 0.1, 0.3, 0.8],
                             width=1)
ccenters = vp.scene.visuals.Markers(
        pos=eptm.cell_df[eptm.coords].values,
        face_color=[1, 1, 1])

view.add(mesh)
view.add(wire)
# view.add(ccenters)
canvas.show()

app.run()

0

In [8]:
import vispy.io as io

In [9]:
bgeom.update_all(eptm)

In [10]:
eptm.je_df.head()

,trgt,srce,nx,length,face,ny,nz,cell,dx,dy,dz,sub_area,sub_vol
je,,,,,,,,,,,,,
0,25,15,0.0625,0.559017,14,0.0625,0.125,0,0.0,0.5,-0.25,0.076547,0.002604
1,17,25,0.0625,0.559017,14,0.0625,0.125,0,-0.5,0.0,0.25,0.076547,0.002604
2,15,3,0.0625,0.559017,4,-0.0625,0.125,0,0.0,0.5,0.25,0.076547,0.002604
3,22,15,0.0625,0.559017,4,-0.0625,0.125,0,-0.5,0.0,0.25,0.076547,0.002604
4,1,22,0.0625,0.559017,4,-0.0625,0.125,0,0.0,-0.5,-0.25,0.076547,0.002604


In [11]:
io.write_mesh('test.obj', vertices=vertices, faces=faces,
              normals=eptm.je_df[['nx', 'ny', 'nz']].values,
              texcoords=None, overwrite=True)